In [80]:
import pandas as pd
import re
import concurrent.futures

In [81]:
# Extract the relevant audio features from each song
import spotipy
from spotipy.oauth2 import SpotifyOAuth
CLIENT_ID = '9f951db530d8462fbedfd75507b90cbf'
CLIENT_SECRET = '3f38813ebdb24d0caa8db79c5a169862'
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, redirect_uri='http://localhost:5000/callback', client_id=CLIENT_ID, client_secret=CLIENT_SECRET))

# Audio Features
artist_name = []
track_id = []
artist_uri = []
for sp_range in ['short_term']:
    results = sp.current_user_top_tracks(time_range=sp_range, limit=50)
    for i, item in enumerate(results['items']):
        artist_uri.append(item['artists'][0]['id'])
        artist_name.append(item['artists'][0]['name'])
        track_id.append(item['id'])

In [82]:
user_songs = pd.DataFrame({
    'artist_name': artist_name,
    'track_id': track_id,
    'artist_uri': artist_uri
})

In [83]:
def extract_track_features(track_id):
    features = sp.audio_features(track_id)[0]

    # Add in track poularity:
    track_dict = sp.track(track_id)
    features['track_pop'] = track_dict['popularity']
    features['track_name'] = track_dict['name']

    # Add in artist genres
    artist = sp.track(track_id)['artists'][0]['id']
    artist_dict = sp.artist(artist)
    artist_genres = artist_dict['genres']
    features['artist_pop'] = artist_dict['popularity']
    if artist_genres:
        features['genres'] = " ".join([re.sub(' ','_',i) for i in artist_genres])
    else:
        features['genres'] = 'unknown'
    return features

In [84]:
# Iterate through and get a list of features for each song
feature_lst = []
with concurrent.futures.ThreadPoolExecutor() as exectuor:
    track_ids = list(user_songs['track_id'])
    feature_generator = exectuor.map(extract_track_features, track_ids)
    
    for feature in feature_generator:
        feature_lst.append(feature)

In [85]:
# Making the features a df
feature_df = pd.DataFrame(feature_lst)

In [86]:
# Merging the original df with features
new_df = pd.merge(user_songs, feature_df, left_on='track_id', right_on='id')
new_df.head()

,artist_name,track_id,artist_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,id,uri,track_href,analysis_url,duration_ms,time_signature,track_pop,track_name,artist_pop,genres
0,Jimin,3Ua0m0YmEjrMi9XErKcNiR,1oSPZhvZMIrWW5I41kPkkY,0.629,0.733,7,-5.445,1,0.0419,0.0025,...,3Ua0m0YmEjrMi9XErKcNiR,spotify:track:3Ua0m0YmEjrMi9XErKcNiR,https://api.spotify.com/v1/tracks/3Ua0m0YmEjrM...,https://api.spotify.com/v1/audio-analysis/3Ua0...,212241,4,93,Like Crazy,82.0,k-pop
1,BTS,5QDLhrAOJJdNAmCTJ8xMyW,3Nrfpe0tUJi4K4DXYWgMUX,0.746,0.765,6,-4.410,0,0.0993,0.0112,...,5QDLhrAOJJdNAmCTJ8xMyW,spotify:track:5QDLhrAOJJdNAmCTJ8xMyW,https://api.spotify.com/v1/tracks/5QDLhrAOJJdN...,https://api.spotify.com/v1/audio-analysis/5QDL...,199054,4,83,Dynamite,86.0,k-pop k-pop_boy_group pop
2,Jung Kook,5Avril3IZ26DPVFHbJX8o6,6HaGTQPmzraVmaVxvz6EUc,0.776,0.831,11,-3.975,1,0.0457,0.3160,...,5Avril3IZ26DPVFHbJX8o6,spotify:track:5Avril3IZ26DPVFHbJX8o6,https://api.spotify.com/v1/tracks/5Avril3IZ26D...,https://api.spotify.com/v1/audio-analysis/5Avr...,185295,4,84,Seven (feat. Latto) (Clean Ver.),88.0,k-pop
3,DPR IAN,6syar8JKCt3R9ZBl11zmgI,2o8gT0fQmFxGNbowbdgeZe,0.806,0.735,7,-4.833,0,0.0491,0.2810,...,6syar8JKCt3R9ZBl11zmgI,spotify:track:6syar8JKCt3R9ZBl11zmgI,https://api.spotify.com/v1/tracks/6syar8JKCt3R...,https://api.spotify.com/v1/audio-analysis/6sya...,187227,4,69,So Beautiful,68.0,k-rap
4,NewJeans,5sdQOyqq2IDhvmx2lHOpwd,6HvZYsbFfjnjFrWF950C9d,0.776,0.817,5,-6.018,0,0.0748,0.1800,...,5sdQOyqq2IDhvmx2lHOpwd,spotify:track:5sdQOyqq2IDhvmx2lHOpwd,https://api.spotify.com/v1/tracks/5sdQOyqq2IDh...,https://api.spotify.com/v1/audio-analysis/5sdQ...,154667,4,90,Super Shy,82.0,k-pop k-pop_girl_group


In [87]:
new_df.to_csv('../data/raw_user_data.csv')